In [13]:
import nltk.data
import operator
import string
import math
from nltk.corpus import stopwords
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
stop = stopwords.words('english') + list(string.punctuation) + list(string.digits)


## Asociación de palabras

### Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario (``<DOCNO>``: ``<TEXT>``)

In [2]:
apCorpus = {}
with open('ap/ap.txt', 'r') as ap:
    nextDoc = ap.readline()
    while nextDoc:
        docNumber = ap.readline().split(' ')[1]
        ap.readline()
        text = ap.readline().strip()
        ap.readline()
        ap.readline()
        apCorpus[docNumber] = text
        nextDoc = ap.readline()


### Calcular el tamaño del vocabulario

In [3]:
apWords = set()
apWordsFrequencies = {}
apCorpusSize = 0
for _, text in apCorpus.iteritems():
    sentences = sent_detector.tokenize(text)
    for sentence in sentences:
        tokenList = [i for i in nltk.word_tokenize(sentence.lower()) if i not in stop]
        apCorpusSize += len(tokenList)
        tokens = set(tokenList)
        apWords |= tokens
        for word in tokens:
            if not word in apWordsFrequencies:
                apWordsFrequencies[word] = 0
            apWordsFrequencies[word] += sentence.count(word)


In [4]:
print 'Tamaño del vocabulario:', len(apWords)
print 'Tamaño del corpus:', apCorpusSize

Tamaño del vocabulario: 42627
Tamaño del corpus: 580913


### Para las 500 palabras con más apariciones, calcular el par más asociado según la medida presentada

In [5]:
apTop500Freq = []
for i, (word, count) in enumerate(sorted(apWordsFrequencies.iteritems(), key=operator.itemgetter(1), reverse = True)):
    if i >= 500:
        break
    apTop500Freq.append(word)
print apTop500Freq

['said', '``', "''", "'s", 'would', 'percent', 'year', 'one', "n't", 'people', 'million', 'also', 'two', 'government', 'last', 'years', 'could', 'new', 'first', 'time', 'officials', 'state', 'billion', 'today', 'police', 'president', 'told', 'three', 'day', 'since', 'week', 'company', 'made', 'say', 'market', 'report', 'country', 'military', 'group', 'federal', 'many', 'members', 'reported', 'get', 'political', 'including', 'make', 'former', 'news', 'going', '10', 'work', 'home', 'spokesman', 'think', 'like', 'says', 'back', 'official', 'next', 'month', 'way', 'go', 'take', 'court', 'still', 'prices', "'re", 'called', 'four', 'found', 'part', 'much', 'nation', 'campaign', 'money', 'use', 'may', 'law', '...', 'program', 'case', 'plan', 'used', 'days', 'office', 'city', 'months', 'meeting', 'support', 'end', 'five', 'late', 'economic', 'help', 'several', 'trade', 'ago', 'public', 'men', 'later', 'world', 'man', 'expected', 'killed', 'least', 'began', 'workers', 'early', 'us', 'party', 'm

In [17]:
window = 5
def f2(x,y):
    res = 0
    for _, text in apCorpus.iteritems():
        words = [i for i in nltk.word_tokenize(text.lower()) if i not in stop]
        for i,word in enumerate(words):
            if word == x:
                for j in range(1, window + 1):
                    if (i + j < len(words)):
                        if words[i+j] == y:
                            res += 1
    return float(res)

def f1(x):
    return float(apWordsFrequencies[x])
    
def I(x, y):
    p_x_y = f2(x,y) / apCorpusSize
    p_x = f1(x) / apCorpusSize
    p_y = f1(y) / apCorpusSize
    return math.log((p_x_y / (p_x * p_y)), 2.0)

In [18]:
max_pair_value = 0
max_pair = None
for i in range(500):
    print i
    for j in range(i, 500):
        new_I = I(apTop500Freq[i], apTop500Freq[j])
        if max_pair_value < new_I:
            max_pair_value = new_I
            max_pair = (apTop500Freq[i], apTop500Freq[j])
print max_pair, max_pair_value

KeyboardInterrupt: 